In [1]:
import numpy as np

from sklearn.metrics.pairwise import pairwise_kernels, pairwise_distances

import seaborn as sns


In [16]:
def compute_constant(d, sigma):
    denom = np.power(2 * np.pi, d / 2.) * np.power(sigma, d/2)
    return 1 / denom


def compute_kernel(X, sigma):
    """
    X : (n_samples, n_dim)
    sigma : float
    """
    
    d = X.shape[1]
    constant = compute_constant(d, sigma)
    
    kern = pairwise_kernels(X, metric='rbf', gamma=sigma / 2) * constant
    
    return kern

def compute_distance(dx, dy, i, j, k, l):
    withinx = dx[i,j] + dx[k,l] - dx[i,k] - dx[j,l]
    withiny = dy[i,j] + dy[k,l] - dy[i,k] - dy[j,l]
    
    return withinx*withiny

In [44]:
def compute_cov(X, Y, Z, sigma=1, power = 1):
    dx = pairwise_distances(X) ** power
    dy = pairwise_distances(Y) ** power
    dz = compute_kernel(Z, sigma)
    
    n = X.shape[0]
    cov = np.zeros(n)

    for u in range(n):
        for i in range(n):
            for j in range(n):
                for k in range(n):
                    for l in range(n):
                        dijkl = compute_distance(dx, dy, i, j, k, l)
                        dijlk = compute_distance(dx, dy, i, j, l, k)
                        dilkj = compute_distance(dx, dy, i, l, k, j)
                        d = dijkl + dijlk + dilkj

                        cov[u] += d * dz[i,u] * dz[j, u] * dz[k, u] * dz[l, u]

        cov[u] /= np.power(n, 4)
        
        
    return cov.mean()


def compute_cor(X, Y, Z, sigma=1):
    cov_xy = compute_cov(X, Y, Z, sigma)
    cov_xx = compute_cov(X, X, Z, sigma)
    cov_yy = compute_cov(Y, Y, Z, sigma)

    return cov_xy / np.sqrt(cov_xx * cov_yy)

In [37]:
X = np.loadtxt("../../src/x.csv", skiprows=1, delimiter=',')
Y = np.loadtxt("../../src/y.csv", skiprows=1, delimiter=',')
Z = np.loadtxt("../../src/z.csv", skiprows=1, delimiter=',')

In [45]:
compute_cov(X, Y, Z, )

6.761120434936672e-07

In [49]:
def example1(n):
    mean = np.array([0, 0, 0])
    cov = np.array(
        [1, 0.36, 0.6],
        [0.36, 1, 0.6],
        [0.6, 0.6, 1]
    )
    
    X = np.random.multivariate_normal(mean, cov, size=n)
    Y = np.random.multivariate_normal(mean, cov, size=n)
    Z = np.random.multivariate_normal(mean, cov, size=n)
    
    return X, Y, Z